# Topic Modeling for COVID19
COVID 19 has been the biggest pandemic people have seen in the recent times. It almost feels like one of those apocalyptic movies in real life. People hiding in their homes trying to save themselves from the infection. Some brave souls trying to find a better destination to survive this pandemic. With so much happening around the world, I have one question. **What are people around the world thinking about COVID 19?**

In this notebook, we will try to answer the above question using Topic Modeling. Let's explore the various topics people are talking about Corona Virus Disease 2019(COVID-19) in Twitter.

In [2]:
# Import all required libraries
import os
import json
import requests
import re

# Twitter data collection library
import tweepy as tw

# Data processing libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Loading Gensim and nltk libraries
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import nltk
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])
#Other stop_words: gensim.parsing.preprocessing.STOPWORDS


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/deepakawari/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Configure notebook display to show data from pandas dataframe more clearly.
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',100)
pd.set_option('display.max_colwidth',800)

# 1. Gather the textual data for Topic Modeling
To begin with the topic modeling we need the textual data. The textual data for what people are talking about COVID 19 can be pulled from many places such as social media, news articles, web scraping etc. In this notebook, we'll download the data from Twitter. Tweepy is an amazing library to pull data from twitter using your Twitter Developer Account. 

## Extracting tweets from Twitter API:
To load the data from Twitter using Tweepy API, you'll have to create Developer account with Twitter. Then download the credentials to authenticate using Tweepy API. **Please do not share these credentials with anybody else.**
* Here is the link to [apply for twitter developer access](https://developer.twitter.com/en/apply-for-access)
* You can follow the below code to use Tweepy API to authenticate and load the data. Here is the [Tweepy Documentation for reference](http://docs.tweepy.org/en/latest/) 

In [75]:
'''
LoadFromTwitter - 
    If true, pull the latest set of tweets from Twitter using the Tweepy library.
    If false, load the data from the datafile '../data/tweets.csv' if it exists, 
    otherwise load the tweets from Twitter using the Tweepy library.
    Set the LoadFromTwitter to True if you want to override loading the tweets afresh from twitter.
'''
LoadFromTwitter = False

fileName = '../data/tweets.csv'
tweetsDF = None

# Load the data
if os.path.exists(fileName) and not LoadFromTwitter:
    tweetsDF = pd.read_csv(fileName)
else:
    from TwitterDevSecrets import getTwitterDevCreds
    consumer_key, consumer_secret, access_token, access_secret = getTwitterDevCreds()

    auth = tw.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    # Set the wait_on_rate_limit and wait_on_rate_limit_notify to True
    # wait_on_rate_limit – 
    #    Whether or not to automatically wait for rate limits to replenish
    # wait_on_rate_limit_notify – 
    #    Whether or not to print a notification when Tweepy is waiting 
    #    for rate limits to replenish
    api = tw.API(
        auth, 
        wait_on_rate_limit=True, 
        wait_on_rate_limit_notify=True)

    # Define the search term and the date_since date as variables
    search_words = "#covid OR #covid19 OR #COVID OR #COVID19 OR #ncov OR #corona OR #coronaviru"
    date_since = "2020-05-16"
    
    # Read the tweets
    tweets = tw.Cursor(api.search, 
                   q=search_words,
                   lang="en",
                   since=date_since)

    # extract the data in pandas dataframe
    # Other parameters: tweet.user.screen_name, retweet_counts, favorite_counts
    tweetsDF = pd.DataFrame()
    for tweet in tweets.items(10000):
        id = tweet.id
        text = tweet.text
        loc = tweet.user.location
        tweetsDF = tweetsDF.append({'Id':id, 'Text':text, 'Location':loc},ignore_index=True)
    
    tweetsDF['index'] = tweetsDF.index
    
    # Save the new set of tweets in the file.
    tweetsDF.to_csv(fileName,index=False)



Let's see how doest he textual data look like.

In [76]:
tweetsDF.Text[1]

'RT @Carol_D_Johnson: Thank you nurses for helping to keep us healthy  ❤ #COVID19 \n#StayHomeSaveLives \n#coronavirus https://t.co/HGv0HfuTgt'

# 2. Data preprocessing
As you can see from the above text, a tweet contains a lot of textual data which probably doesn't contain any useful informaiton for Topic Modeling. So, these tweets needs to be processed to extract only useful textual data for further analysis. We will perform the following data processing steps:

* Tweet Preprocessing:
> * Remove the leading **RT** - RT indicates that the user is re-posting someone else's tweet. We can remove this token.
> * Remove the references to other accounts. The other accounts are usually referenced with '@' symbol.
> * Remove urls mentioned in the tweets.

* Generic text preprocessing:
> * **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
> * Remove words that have fewer than 3 characters.
> * Remove all **stopwords**. [Stop words](https://en.wikipedia.org/wiki/Stop_words) usually do not contain any usual information. As such these words are generally removed from the text in the preprocessing stage. 
> * **Lemmatize** the words: words in third person are changed to first person and verbs in past and future tenses are changed into present.  
> Lemmatization, unlike Stemming, reduces the inflected words properly ensuring that the root word belongs to the language. In Lemmatization root word is called Lemma. A lemma (plural lemmas or lemmata) is the canonical form, dictionary form, or citation form of a set of words. 
>> **WordnetLemmatizer**: uses lookup table from nltk wordnet corpus to lookup the lemma to return a valid language lemma.
> * **Stem** the Words: words are reduced to their root form.  
> Stemming is the process of reducing inflection in words to their root forms such as 
mapping a group of words to the same stem even if the stem itself is not a valid word 
in the Language.
>> **PorterStemmer**: is known for simplicity and ease. The algorithm does not follow linguistics rather a set of rules that are applied in phases (step by step) to generate stems. This is the reason why PorterStemmer does not often generate stems that are actual English words.  
>> **SnowballStemmer**: One can generate their own set of rules for any language. Python nltk introduced SnowballStemmers that are used to create non-English Stemmers!  
>> **LancasterStemmer**: is simple, but heavy stemming due to iterations and over-stemming may occur. Over-stemming causes the stems to be not linguistic, or they may have no meaning.
    

In [96]:
# Perform data preprocessing for all tweets.
stemmer = SnowballStemmer("english")

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def tweet_cleanup(text):
    # Remove the leading RT from the tweet
    text = text.replace('RT','')
    # Remove the references to the account names starting with '@'
    text = re.sub(r'(@[a-zA-Z]*)','',text)
    # Remove the urls in the tweet.
    text = re.sub(r'((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)','',text)
    
    return text
  
# Clean up the tweets and then Tokenize and lemmatize
def preprocess(text, stop_words=stop_words):
    result=[]
    text = tweet_cleanup(text)
    for token in gensim.utils.simple_preprocess(text) :
        if token not in stop_words and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result



In [97]:
# Test the preprocessing step on a sample tweet
tweet_num = 0
sampleTweet = tweetsDF[tweetsDF['index'] == tweet_num].Text.iloc[0]

print("Original tweet: ")
words = []
for word in sampleTweet.split():
    words.append(word)
print(words)
print("\n\nPreprocessed tweet: ")
print(preprocess(sampleTweet))

Original tweet: 
['RT', '@ALPublicHealth:', 'State', 'Health', 'Officer', 'Dr.', 'Scott', 'Harris', 'has', 'issued', 'a', 'stay', 'at', 'home', 'order', 'and', 'strict', 'quarantine', 'requirements.', 'Read', 'our', 'full…']


Preprocessed tweet: 
['state', 'health', 'offic', 'scott', 'harri', 'issu', 'stay', 'home', 'order', 'strict', 'quarantin', 'requir', 'read', 'full']


In [98]:
# Preprocess all tweets and generate a new processed tweet text dataset.
processed_tweets = tweetsDF['Text'].map(preprocess)
processed_tweets[:10]

0        [state, health, offic, scott, harri, issu, stay, home, order, strict, quarantin, requir, read, full]
1                                                      [thank, nurs, help, keep, healthi, covid, coronavirus]
2                                                                        [togetherapart, slow, spread, covid]
3                    [smoker, greater, risk, contract, coronavirus, elli, cannon, say, equal, risk, contract]
4                                      [video, model, scan, show, extent, covid, damag, lung, tissu, stayhom]
5                      [leader, hous, parti, caucus, arizona, andi, bigg, think, spread, covid, much, possib]
6                                                                                     [covid, test, administ]
7           [keep, think, master, public, health, write, doctor, dissert, global, effort, tackl, aid, pandem]
8    [ceylonblacktea, rich, theaflavin, help, increas, human, immun, covid, srilankatea, industri, successfu]
9         

# 3. Text representation
Computers don't understand natural language texts. Text is a mere sequence of letters for computers. While its still difficult for computer to understand what the sequence of letters mean, language is way more complicated than that. For an example, let us consider an idiom "Kicked the bucket". You know where I am going right? When I first heard that phrase as a kid I thought it meant someone was actually kicking a bucket. That's fun! But, when I realized that it meant someone died, it was no more fun! So, natural language is hard and computers don't understand it. 

Computers love numbers. At the core, computers perform their operations on numbers. So, it'd be good to represent the natural language text with numbers for computer algorithms to process easily. In the below section, we'll explore two different models for text representation namely Bag of words and TF-IDF.

# 3.1: Bag of words on the dataset
Create a dictionary of words present in the preprocessed_tweets dataset. Gensim offers a great api for the same. This dictionary assigns a numerical id to each word so that you can work on the number representations of the word. This makes the data processing very easy than working on strings. 

Then create a corpus of Bag of words where words are represented by their numerical ids along with the frequency of occurence of that word in the tweet for further processing.

In [9]:
dictionary = gensim.corpora.Dictionary(processed_tweets)

# Create Corpus: Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in processed_tweets]

In [10]:
# Check the id to word mapping from the dictionary created above
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 5:
        break

0 full
1 harri
2 health
3 home
4 issu
5 offic


Since the text corpus is very huge and sparse, we should try to minimize the amount of text being used for modeling. For this reason, let us remove very rare and very common words. Gensim dictionary object provides a good api to perform this operation.
- words appearing less than 15 times
- words appearing in more than 10% of all documents

Then convert it into bag of word corpus with very rare and very common wordsd filtered out.

In [11]:
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_tweets]

# Test the Bag of Words representation of the tweet --> (token_id, token_count)
bow_corpus[tweet_num]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]

In [12]:
# Preview BOW for our sample preprocessed tweet
bow_tweet_0 = bow_corpus[tweet_num]

for i in range(len(bow_tweet_0)):
    print("Word {} (\"{}\") appears {} time.".format(bow_tweet_0[i][0], 
                                                     dictionary[bow_tweet_0[i][0]], 
                                                     bow_tweet_0[i][1]))

Word 0 ("health") appears 1 time.
Word 1 ("home") appears 1 time.
Word 2 ("order") appears 1 time.
Word 3 ("state") appears 1 time.
Word 4 ("stay") appears 1 time.


# 3.2: TF-IDF on the data set
TFIDF, short for term frequency–inverse document frequency, is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus. Summing the Tf-idf of all possible terms and documents recovers the mutual information between documents and term taking into account all the specificities of their joint distribution.

TF (Term Frequency) - number of times a term occurs in a document.  
IDF (Inverse Document Frequency) diminishes the weight of terms that occur very frequently in the document set and increases the weight of terms that occur rarely.

In [13]:
# Create tf-idf model object using models.TfidfModel
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)

# Apply transformation to the entire corpus
tfidf_corpus = tfidf[bow_corpus]

# Test the tf-idf representation of the sample tweet. Each word is represented by (token_id, tf-idf score).
tfidf_corpus[tweet_num]

[(0, 0.40009295170061265),
 (1, 0.44472924895798494),
 (2, 0.45253501051552114),
 (3, 0.47433139975516847),
 (4, 0.4608289406979316)]

In [14]:
# Preview TF-IDF for our sample preprocessed tweet
tfidf_tweet_0 = tfidf_corpus[tweet_num]

for i in range(len(tfidf_tweet_0)):
    print("Word {} (\"{}\") TF-IDF score: {}.".format(tfidf_tweet_0[i][0], 
                                                     dictionary[tfidf_tweet_0[i][0]], 
                                                     tfidf_tweet_0[i][1]))

Word 0 ("health") TF-IDF score: 0.40009295170061265.
Word 1 ("home") TF-IDF score: 0.44472924895798494.
Word 2 ("order") TF-IDF score: 0.45253501051552114.
Word 3 ("state") TF-IDF score: 0.47433139975516847.
Word 4 ("stay") TF-IDF score: 0.4608289406979316.


# 4: Topic modeling using LDA
Topic modeling is a statistical model to discover the abstract topics in a collection of documents. Probabilistic Latent Semantic Analysis(PLSA) is one of the ealiest models for topic modeling. Latent Dirichlet Allocation(LDA) is the most common topic model algorithm in use today which is a generalization of PLSA. 

LDA introduces sparse Dirichlet prior distributions over document-topic and topic-word distributions. This algorithm tries to model the intuition that each document has different abstract topics and that each topic is generalized by a small number of words.

In this section we'll be building the topic models using LDA for both text representations developed above. 


## 4.1: LDA using Bag of Words
In this section, we'll train the LDA model to generate topic clusters using Bag of Words (BOW) corpus. The LDA algorithm requires a few inputs to build the clusters. The main parameter it requires is the number of clusters we want the model to group the tweets into. But how do we identify the number of topics? Let's start with some intuitive value like 5 or 10. We'll then tune this hyper parameter in the next section. 

In [82]:
# Train the lda model using gensim.models.LdaMulticore on Bag of word corpus
lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=5, 
                                       id2word = dictionary, 
                                       passes = 2, 
                                       workers=2)

In [83]:
# Explore the words occuring in that topic and its relative weight
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.050*"mask" + 0.047*"help" + 0.041*"send" + 0.039*"time" + 0.039*"today" + 0.038*"thank" + 0.037*"need" + 0.037*"say" + 0.037*"coronavirus" + 0.032*"right"


Topic: 1 
Words: 0.089*"coronavirus" + 0.035*"everyon" + 0.034*"help" + 0.034*"affect" + 0.031*"make" + 0.031*"pleas" + 0.027*"protect" + 0.027*"poor" + 0.024*"face" + 0.024*"discrimin"


Topic: 2 
Words: 0.053*"work" + 0.052*"home" + 0.049*"caus" + 0.048*"trut" + 0.048*"father" + 0.048*"natur" + 0.048*"racism" + 0.048*"speak" + 0.047*"poverti" + 0.047*"murder"


Topic: 3 
Words: 0.062*"like" + 0.042*"public" + 0.038*"never" + 0.033*"amaz" + 0.031*"live" + 0.030*"peopl" + 0.029*"often" + 0.029*"work" + 0.028*"gullibl" + 0.028*"narrat"


Topic: 4 
Words: 0.077*"case" + 0.060*"test" + 0.049*"peopl" + 0.043*"health" + 0.028*"human" + 0.028*"keep" + 0.024*"april" + 0.024*"death" + 0.023*"state" + 0.023*"make"




## 4.2: LDA using TF-IDF
TF-IDF intends to reflect on the importance of each word in the tweet amongst other tweets. Thus it tries to create a better model instead of using mere Term Frequency as in Bag of words model. However, for TF-IDF to work it needs to have a good size of text in each document. However, tweet is usually very small in size. Thus, most of the times each word ends up being mentioned only once. Thus, TF-IDF doesn't work better for short texts. However, let's train the model and evaluate the performance and see how does it perform.

In [84]:
# Train lda model using corpus_tfidf
lda_model_tfidf = gensim.models.LdaMulticore(tfidf_corpus, 
                                             num_topics=5, 
                                             id2word = dictionary, 
                                             passes = 2, 
                                             workers=4)

In [85]:
# Explore the words occuring in that topic and its relative weight
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.057*"test" + 0.051*"make" + 0.045*"pandem" + 0.040*"die" + 0.033*"help" + 0.033*"ask" + 0.031*"hospit" + 0.031*"confirm" + 0.029*"patient" + 0.027*"equip"


Topic: 1 Word: 0.051*"case" + 0.033*"caus" + 0.032*"natur" + 0.031*"father" + 0.031*"speak" + 0.031*"poverti" + 0.031*"trut" + 0.031*"racism" + 0.031*"murder" + 0.031*"report"


Topic: 2 Word: 0.040*"health" + 0.039*"order" + 0.034*"thank" + 0.033*"medic" + 0.033*"time" + 0.029*"stay" + 0.029*"coronavirus" + 0.028*"take" + 0.027*"state" + 0.026*"keep"


Topic: 3 Word: 0.085*"coronavirus" + 0.079*"fight" + 0.048*"china" + 0.035*"india" + 0.032*"support" + 0.031*"enough" + 0.030*"work" + 0.027*"protect" + 0.024*"live" + 0.023*"make"


Topic: 4 Word: 0.073*"peopl" + 0.049*"spread" + 0.044*"mask" + 0.040*"need" + 0.037*"think" + 0.036*"trump" + 0.033*"say" + 0.032*"pleas" + 0.031*"home" + 0.028*"week"




# 5. Model evaluation
In the previous section, we created the topic models using LDA. But how do we evaluate if the clustering is good at all? Topic cluster is a statistical tool that imposes probability distribution over words. There are so many ways to evaluate the models. Some of the ways to evaluate the models are mentioned below.
* By visualization, we can generate an intuition behind the quality of the topics by visualizing. 
> * We can try visualizing the clusters in intertopic distance mapping via multi-dimensional scaling. In this 2 dimensional map, if the clusters are overlapping then the cluster segregation wasn't good. On the otherhand, if the clusters were separated clearly, that'd be a good model. We'll explore this more when we visualize the clusters using PyLDAVis in the further sections.
> * We could also try running the sample examples through the model and see how well the sample examples get classified into topics and how well they match the clusters. 

* Using performance evaluation metrics
We could also try using different clustering evaluation metrics. There are two types of metrics intrinsic and extrinsic metrics. 
> * Perplexity is a good intrinsic performance evaluation metric. It measures how well the probability distribution predicts a sample. A low perplexity indicates the probablity distribution is good at predicting the sample.
> * However, coherence measures have shown to be better metrics for topic cluster evaluations. So, in this section, we use coherence measures to quantify the quality of the clusters and the probability distribution of the words. 


## 5.1. Evaluation by human validation
In this section, we'll cluster a sample tweet and see how well does this clustering matches the overall topics generated above.

### 5.1.1: Human validation for Bag of words model
Classify a sample tweet into the topics and then evaluate if the general topic matches with the tweet better than other topics.

In [21]:
# Our test tweet is 
print('Our test tweet is: {}: {}'.format(tweet_num, [dictionary[word[0]] for word in bow_corpus[tweet_num]]))

for index, score in sorted(lda_model[bow_corpus[tweet_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {} : {}".format(score, index, lda_model.print_topic(index, 10))) 

Our test tweet is: 0: ['health', 'home', 'order', 'state', 'stay']

Score: 0.5977810025215149	 
Topic: 2 : 0.044*"health" + 0.034*"case" + 0.034*"keep" + 0.033*"pandem" + 0.026*"time" + 0.025*"death" + 0.024*"care" + 0.023*"human" + 0.023*"like" + 0.021*"trump"

Score: 0.34373247623443604	 
Topic: 0 : 0.043*"work" + 0.038*"natur" + 0.035*"caus" + 0.034*"father" + 0.034*"murder" + 0.033*"speak" + 0.033*"trut" + 0.033*"poverti" + 0.033*"racism" + 0.030*"mask"

Score: 0.05848647654056549	 
Topic: 1 : 0.070*"coronavirus" + 0.065*"peopl" + 0.050*"test" + 0.041*"help" + 0.029*"affect" + 0.025*"everyon" + 0.021*"hospit" + 0.020*"fight" + 0.020*"poor" + 0.019*"either"


The sample tweet is classified to Topic 0 with 88% score. Topic 0 in Bag of word model was centered around self quarantining. The sample tweet matches with this topic. We could try evaluating more tweets manually. Seems like the BOW based LDA model worked well.    

### 5.1.2 Human validation for TF-IDF model

In [22]:
# Our test tweet is 
print('Our test tweet is: {}: {}'.format(tweet_num, [dictionary[word[0]] for word in tfidf_corpus[tweet_num]]))

for index, score in sorted(lda_model_tfidf[tfidf_corpus[tweet_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {} : {}".format(score, index, lda_model.print_topic(index, 10)))

Our test tweet is: 0: ['health', 'home', 'order', 'state', 'stay']

Score: 0.6827797293663025	 
Topic: 2 : 0.044*"health" + 0.034*"case" + 0.034*"keep" + 0.033*"pandem" + 0.026*"time" + 0.025*"death" + 0.024*"care" + 0.023*"human" + 0.023*"like" + 0.021*"trump"

Score: 0.1886352002620697	 
Topic: 1 : 0.070*"coronavirus" + 0.065*"peopl" + 0.050*"test" + 0.041*"help" + 0.029*"affect" + 0.025*"everyon" + 0.021*"hospit" + 0.020*"fight" + 0.020*"poor" + 0.019*"either"

Score: 0.128585085272789	 
Topic: 0 : 0.043*"work" + 0.038*"natur" + 0.035*"caus" + 0.034*"father" + 0.034*"murder" + 0.033*"speak" + 0.033*"trut" + 0.033*"poverti" + 0.033*"racism" + 0.030*"mask"


As can be seen above, the sample tweet is split between topics 0 with 49% score and topic 1 with 38% score. As we saw in the section 4.2.1, Topic 0 was centered around President Trump's announcement around COVID 19 and Topic 1 around Quarantine and fight the spread of COVID-19. 

The sample tweet matches well with Topic 1. However, the confidence of this clustering is low compared to 88% confidence we saw for Bag of Words model. 

The TF-IDF modeling didn't have good confidence in the classification. This was expected as TF-IDF doesn't work good for short text documents. 

## 5.2 Coherence measures
The topic coherence measures scores a single topic by computing the semantic similarity between the top words in that topic. We can then average the scores of each topic to get the overall coherence measure for the model.

There are different coherence measures. In the below evaluation we'll be using the C_v measure. C_v measure is based on a sliding window, a one-set segmentation of the top words and an indirect confirmation measure that uses normalized pointwise mutual information (NPMI) and the cosinus similarity.

In [23]:
from gensim.models import CoherenceModel

# Compute Coherence Score for BOW based LDA model
cv_model_bow = CoherenceModel(model=lda_model, texts=processed_tweets, dictionary=dictionary, coherence='c_v')
cv_bow_overall = cv_model_bow.get_coherence()
cv_bow_pertopic = cv_model_bow.get_coherence_per_topic()
print('\nCoherence Score: {0}\nPer Topic: {1}'.format(cv_bow_overall , cv_bow_pertopic))


Coherence Score: 0.4304417242985787
Per Topic: [0.4493676045869341, 0.33174836101253, 0.510209207296272]


In [24]:
# Compute Coherence Score for TF-IDF based LDA model
cv_model_tfidf = CoherenceModel(model=lda_model_tfidf, texts=processed_tweets, dictionary=lda_model_tfidf.id2word, coherence='c_v')
cv_tfidf_overall = cv_model_tfidf.get_coherence()
cv_tfidf_pertopic = cv_model_tfidf.get_coherence_per_topic()
print('\nCoherence Score: {0}\nPer Topic: {1}'.format(cv_tfidf_overall, cv_tfidf_pertopic))



Coherence Score: 0.5277570239510211
Per Topic: [0.5811252711244042, 0.46330507155149236, 0.5388407291771666]


Based on the coherence measures, it seems that the tf-idf model seemed to have gathered the topics based on better semantic similarity between the words. This was contradictory to what we saw in the previous section on human validation of the topics. However, the previous measure was done on a few sample tweets. 

# 6. Hyper parameter tuning 
Now that we have established the modeling and performance evaluation methods, let's try to tune the Number of Topics hyper parameter. We'll define a range for the number of topics. Then we'll train a model for each value for the hyper parameter. We'll also compute the coherence score and jot it down. At the end we'll plot the scores to see what's the best for the Number of Topics hyper parameter.

We'll repeat the same for both BOW and TF-IDF models and compare the scores to choose the best model.

The LDA model has more hyper-parameters namely alpha and beta values. We could tune the alpha and beta hyperparameters also in the similar fashion. 

For parameter tuning, sklearn exposes a GridSearchCV api that can be configured easily with ranges for different hyper parameters. For this, we can use the LDA_Transform model exposed by gensim. However, that approach uses the default log_likelihood score for tuning the hyper parameters. Since we decided to use coherence scores, we'll tune the hyper parameters in a loop instead of using GridSearchCV.

In [91]:
def compute_coherence_values(corpus, dictionary, num_topics):
    lda_model = gensim.models.LdaMulticore(corpus, 
                                       num_topics=num_topics, 
                                       id2word = dictionary, 
                                       passes = 2, 
                                       workers=2)
        
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_tweets, dictionary=dictionary, coherence='c_v')
    
    return lda_model, coherence_model_lda.get_coherence()

In [92]:
import numpy as np
import tqdm

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Models
model_sets = [bow_corpus, tfidf_corpus]
model_title = ['BOW', 'TF-IDF']

model_results = pd.DataFrame(columns=['Model','Num_Topics','Coherence'])

pbar = tqdm.tqdm(total=len(model_sets)*len(topics_range))
    
if 1 == 1:
  # iterate through validation corpuses
  for i in range(len(corpus_sets)):
    # iterate through number of topics
    for k in topics_range:
        # get the coherence score for the given parameters
        model, cv = compute_coherence_values(corpus=model_sets[i], dictionary=dictionary, num_topics=k)
        
        # Save the model results
        results = {'Model':model_title[i]
                   ,'Num_Topics':k
                   ,'Coherence':cv}
        
        model_results = model_results.append(results , ignore_index=True)
        pbar.update(1)

  model_results.to_csv('lda_tuning_results.csv', index=False)
  pbar.close()



  0%|          | 0/18 [00:00<?, ?it/s]

  6%|▌         | 1/18 [00:01<00:25,  1.49s/it]

 11%|█         | 2/18 [00:02<00:23,  1.47s/it]

 17%|█▋        | 3/18 [00:04<00:21,  1.43s/it]

 22%|██▏       | 4/18 [00:05<00:18,  1.36s/it]

 28%|██▊       | 5/18 [00:06<00:16,  1.31s/it]

 33%|███▎      | 6/18 [00:07<00:15,  1.29s/it]

 39%|███▉      | 7/18 [00:09<00:14,  1.27s/it]

 44%|████▍     | 8/18 [00:10<00:12,  1.28s/it]

 50%|█████     | 9/18 [00:11<00:11,  1.30s/it]

 56%|█████▌    | 10/18 [00:13<00:10,  1.29s/it]

 61%|██████    | 11/18 [00:14<00:09,  1.30s/it]

 67%|██████▋   | 12/18 [00:15<00:07,  1.28s/it]

 72%|███████▏  | 13/18 [00:16<00:06,  1.28s/it]

 78%|███████▊  | 14/18 [00:18<00:05,  1.25s/it]

 83%|████████▎ | 15/18 [00:19<00:03,  1.25s/it]

 89%|████████▉ | 16/18 [00:20<00:02,  1.23s/it]

 94%|█████████▍| 17/18 [00:21<00:01,  1.22s/it]

100%|██████████| 18/18 [00:22<00:00,  1.27s/it]


In [93]:
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

fig = px.line(model_results, x='Num_Topics', y='Coherence',color='Model')
fig.show()

From the above models, seems like both BOW and TF-IDF models with 2 topics performs the best. So, let's train the model with the tuned hyper parameters and use that for further analysis.

In [125]:
lda_model_bow = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=3, 
                                       id2word = dictionary, 
                                       passes = 2, 
                                       workers=2)

lda_model_tfidf = gensim.models.LdaMulticore(tfidf_corpus, 
                                       num_topics=4, 
                                       id2word = dictionary, 
                                       passes = 2, 
                                       workers=2)



# 7. Visualization using PyLDAVis
In this section, we'll visualize the topics generated by the above LDA model using pyLDAvis library. pyLDAvis provides an amazing interactive visualization tool to see how different clusters are generated. It produces the intertopic distance map and shows top relevant terms for each topic amongst other features. 

As mentioned above, we'll visualize the intertopic distance map to see if there is a good segregation of clusters. If there is an overlap between multiple clusters, we'd reduce the number of topics and run the LDA model with reduced number of clusters and visualize again. Once we are satisfied with the cluster segregation in the intertopic distance map, we can start looking into the terms to see what each topic represents.

In [126]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_bow, bow_corpus, dictionary=lda_model.id2word)
vis

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.113176  0.120559       1        1  35.431049
2      0.219116  0.002683       2        1  32.334244
1     -0.105941 -0.123242       3        1  32.234707, topic_info=    Category       Freq         Term      Total  loglift  logprob
42   Default  56.000000         work  56.000000  30.0000  30.0000
40   Default  42.000000        speak  42.000000  29.0000  29.0000
53   Default  72.000000         case  72.000000  28.0000  28.0000
34   Default  40.000000         caus  40.000000  27.0000  27.0000
35   Default  40.000000       father  40.000000  26.0000  26.0000
0    Default  53.000000       health  53.000000  25.0000  25.0000
38   Default  38.000000      poverti  38.000000  24.0000  24.0000
39   Default  38.000000       racism  38.000000  23.0000  23.0000
41   Default  38.000000         trut  38.000000  22.0000  22.0000
36   Default  39.000000       murder  39.000000  21.0000  21.0000
22   Default  71.000000        peopl  71.000000  20.0000  20.0000
13   Default  51.000000         test  51.000000  19.0000  19.0000
37   Default  45.000000        natur  45.000000  18.0000  18.0000
50   Default  31.000000         time  31.000000  17.0000  17.0000
70   Default  42.000000        fight  42.000000  16.0000  16.0000
101  Default  24.000000         face  24.000000  15.0000  15.0000
72   Default  42.000000         make  42.000000  14.0000  14.0000
11   Default  26.000000          say  26.000000  13.0000  13.0000
24   Default  41.000000         mask  41.000000  12.0000  12.0000
3    Default  28.000000        state  28.000000  11.0000  11.0000
14   Default  22.000000       doctor  22.000000  10.0000  10.0000
12   Default  25.000000        think  25.000000   9.0000   9.0000
43   Default  25.000000       govern  25.000000   8.0000   8.0000
6    Default  48.000000         help  48.000000   7.0000   7.0000
29   Default  37.000000         like  37.000000   6.0000   6.0000
18   Default  25.000000        human  25.000000   5.0000   5.0000
104  Default  20.000000        posit  20.000000   4.0000   4.0000
100  Default  19.000000      confirm  19.000000   3.0000   3.0000
73   Default  29.000000        virus  29.000000   2.0000   2.0000
19   Default  17.000000         rich  17.000000   1.0000   1.0000
101   Topic1  23.492628         face  24.673864   0.9885  -3.8037
14    Topic1  21.398134       doctor  22.718824   0.9777  -3.8971
0     Topic1  49.775906       health  53.306252   0.9691  -3.0529
11    Topic1  24.524338          say  26.674429   0.9535  -3.7608
15    Topic1  17.218140       global  18.737322   0.9530  -4.1145
107   Topic1  16.257692        cover  17.720444   0.9514  -4.1719
50    Topic1  28.759945         time  31.586308   0.9438  -3.6014
12    Topic1  23.137199        think  25.559336   0.9380  -3.8190
44    Topic1  16.788557         even  18.676283   0.9310  -4.1397
98    Topic1  13.433551    recommend  15.781865   0.8765  -4.3627
51    Topic1  14.150691       import  16.707415   0.8715  -4.3107
70    Topic1  35.537922        fight  42.152561   0.8669  -3.3898
72    Topic1  34.734211         make  42.157925   0.8439  -3.4127
48    Topic1  22.859636         care  28.549000   0.8153  -3.8310
24    Topic1  32.869621         mask  41.514469   0.8041  -3.4679
60    Topic1  17.714666        china  22.716934   0.7889  -4.0860
58    Topic1  12.146369         life  15.741085   0.7783  -4.4634
77    Topic1  17.261377        equip  22.490934   0.7729  -4.1119
96    Topic1  19.831367      protect  27.117552   0.7247  -3.9732
49    Topic1  21.696056      patient  30.071377   0.7111  -3.8833
33    Topic1  11.131491      support  15.485986   0.7074  -4.5506
9     Topic1  16.548590        thank  23.384338   0.6918  -4.1541
55    Topic1  11.359245       enough  16.549549   0.6613  -4.5304
75    Topic1  14.688077       worker  21.427212   0.6600  -4.2734
16    Topic1  33.231815       pandem  49.225788   0.6447 

In [127]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tfidf, tfidf_corpus, dictionary=lda_model_tfidf.id2word)
vis

/usr/local/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.049807 -0.135289       1        1  26.186216
3     -0.133277  0.136460       2        1  25.618834
1      0.196366  0.054535       3        1  25.289398
2     -0.013282 -0.055706       4        1  22.905554, topic_info=    Category       Freq         Term      Total  loglift  logprob
49   Default  17.000000      patient  17.000000  30.0000  30.0000
53   Default  32.000000         case  32.000000  29.0000  29.0000
16   Default  25.000000       pandem  25.000000  28.0000  28.0000
42   Default  22.000000         work  22.000000  27.0000  27.0000
9    Default  16.000000        thank  16.000000  26.0000  26.0000
37   Default  16.000000        natur  16.000000  25.0000  25.0000
4    Default  16.000000         stay  16.000000  24.0000  24.0000
34   Default  14.000000         caus  14.000000  23.0000  23.0000
6    Default  22.000000         help  22.000000  22.0000  22.0000
36   Default  13.000000       murder  13.000000  21.0000  21.0000
5    Default  42.000000  coronavirus  42.000000  20.0000  20.0000
32   Default  14.000000         want  14.000000  19.0000  19.0000
38   Default  13.000000      poverti  13.000000  18.0000  18.0000
41   Default  13.000000         trut  13.000000  17.0000  17.0000
39   Default  13.000000       racism  13.000000  16.0000  16.0000
35   Default  14.000000       father  14.000000  15.0000  15.0000
3    Default  16.000000        state  16.000000  14.0000  14.0000
10   Default  17.000000       spread  17.000000  13.0000  13.0000
14   Default  12.000000       doctor  12.000000  12.0000  12.0000
15   Default  10.000000       global  10.000000  11.0000  11.0000
40   Default  15.000000        speak  15.000000  10.0000  10.0000
78   Default  20.000000         need  20.000000   9.0000   9.0000
79   Default  10.000000         look  10.000000   8.0000   8.0000
24   Default  21.000000         mask  21.000000   7.0000   7.0000
87   Default  16.000000        pleas  16.000000   6.0000   6.0000
74   Default  10.000000        great  10.000000   5.0000   5.0000
86   Default  12.000000         week  12.000000   4.0000   4.0000
22   Default  34.000000        peopl  34.000000   3.0000   3.0000
27   Default  12.000000     lockdown  12.000000   2.0000   2.0000
50   Default  18.000000         time  18.000000   1.0000   1.0000
32    Topic1  12.990092         want  14.249694   1.2474  -3.4168
74    Topic1   9.573905        great  10.619604   1.2363  -3.7220
100   Topic1   9.143170      confirm  10.980449   1.1568  -3.7680
53    Topic1  25.428717         case  32.428932   1.0968  -2.7451
6     Topic1  17.908781         help  22.871105   1.0954  -3.0957
87    Topic1  13.019506        pleas  16.944826   1.0764  -3.4146
96    Topic1  10.592992      protect  14.633836   1.0168  -3.6208
21    Topic1  11.430941       hospit  15.871010   1.0118  -3.5447
24    Topic1  15.741133         mask  21.993462   1.0055  -3.2247
47    Topic1  10.257074       report  14.338034   1.0050  -3.6530
25    Topic1  10.310595         send  14.487005   0.9999  -3.6478
92    Topic1   8.946719        break  13.913244   0.8984  -3.7897
103   Topic1   5.985230        watch   9.310347   0.8981  -4.1917
97    Topic1   6.159941        polic   9.947659   0.8607  -4.1629
78    Topic1  12.747252         need  20.684496   0.8559  -3.4357
108   Topic1   6.090666       releas   9.944503   0.8497  -4.1742
43    Topic1   9.657081       govern  16.370388   0.8122  -3.7133
77    Topic1   6.942891        equip  12.092203   0.7851  -4.0433
107   Topic1   4.889936        cover   8.777168   0.7550  -4.3938
98    Topic1   4.496140    recommend   8.344910   0.7215  -4.4778
101   Topic1   6.058694         face  11.516433   0.6977  -4.1795
57    Topic1   5.682466       affect  11.174652   0.6637  -4.2436
8     Topic1   6.327899         nurs  12.516233   0.6579  -4.1360
1     Topic1   9.610342         home  19.086058   0.6538  -3.7182
93 

# 8. Dominant Topic for each tweet
So far we built a topic model with tuned hyper parameters. Now, let's find out which topic does each of the tweets belong to. We'll do this by running the tweet through the model. The model would return with a list of topic_id and corresponding Percentage contribution of each topic in the tweet. We'll take the topic with the highest percentage contribution as the dominant topic for the tweet.

In [152]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=None):
    # Init output
    sent_topics_df = pd.DataFrame()
    
    # Let's get the topics and their keywords
    topicsDF = pd.DataFrame()
    for topic_id in range(lda_model_bow.num_topics):
        wp = lda_model_bow.show_topic(topic_id)
        topic_keywords = ", ".join([word for word, prop in wp])
        topicsDF = topicsDF.append(pd.Series([int(topic_id),topic_keywords]),ignore_index=True)
    topicsDF.columns=['Topic_Num','Keywords']
    
    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = sorted(row_list, key=lambda x: (x[1]), reverse=True)
        
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        (topic_num, prop_topic) = row[0]
        sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4)]), ignore_index=True)
    sent_topics_df.columns = ['Topic_Num', 'Perc_Contribution']

    # Merge with topicsDF to add the Keywords from each topic
    sent_topics_df = sent_topics_df.merge(topicsDF, on='Topic_Num', how='left')

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


tweet_topics = format_topics_sentences(ldamodel=lda_model_bow, corpus=bow_corpus, texts=processed_tweets)

# Format
tweet_topics = tweet_topics.reset_index()
tweet_topics.columns = ['Tweet_Num', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
tweet_topics.head(10)

,Tweet_Num,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,1.0,0.6320,"peopl, test, coronavirus, like, help, state, keep, everyon, home, virus","[state, health, offic, scott, harri, issu, stay, home, order, strict, quarantin, requir, read, full]"
1,1,1.0,0.6530,"peopl, test, coronavirus, like, help, state, keep, everyon, home, virus","[thank, nurs, help, keep, healthi, covid, coronavirus]"
2,2,2.0,0.6262,"case, work, coronavirus, speak, natur, caus, father, murder, poverti, racism","[togetherapart, slow, spread, covid]"
3,3,0.0,0.6249,"health, fight, make, pandem, mask, time, say, help, face, think","[smoker, greater, risk, contract, coronavirus, elli, cannon, say, equal, risk, contract]"
4,4,0.0,0.3333,"health, fight, make, pandem, mask, time, say, help, face, think","[video, model, scan, show, extent, covid, damag, lung, tissu, stayhom]"
5,5,0.0,0.5925,"health, fight, make, pandem, mask, time, say, help, face, think","[leader, hous, parti, caucus, arizona, andi, bigg, think, spread, covid, much, possib]"
6,6,1.0,0.6636,"peopl, test, coronavirus, like, help, state, keep, everyon, home, virus","[covid, test, administ]"
7,7,0.0,0.9117,"health, fight, make, pandem, mask, time, say, help, face, think","[keep, think, master, public, health, write, doctor, dissert, global, effort, tackl, aid, pandem]"
8,8,1.0,0.8284,"peopl, test, coronavirus, like, help, state, keep, everyon, home, virus","[ceylonblacktea, rich, theaflavin, help, increas, human, immun, covid, srilankatea, industri, successfu]"
9,9,1.0,0.8221,"peopl, test, coronavirus, like, help, state, keep, everyon, home, virus","[peopl, first, group, hospit, sick, place, infirmari, convert, convent]"


# 9. Most representative tweet for each topic
In this section, let's find out the tweet that matches with the topic strongly. For this, we'll group all tweets by their corresponding dominant topics. For each topic, we'll identify the tweet with highest topic percentage contribution. 

In [156]:
# sent_topics_sorteddf_mallet -> topic_sent
topic_sent = pd.DataFrame()
topic_sent_grpd = tweet_topics.groupby('Dominant_Topic')

for i, grp in topic_sent_grpd:
    topic_sent = pd.concat([topic_sent, grp.sort_values(['Topic_Perc_Contrib'], ascending=False).head(1)], axis=0)

# Reset Index    
topic_sent.reset_index(drop=True, inplace=True)

# Format
topic_sent.columns = ['Tweet_Num','Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]
topic_sent = topic_sent.reindex(columns=['Topic_Num','Tweet_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"])

# Show
topic_sent.head()

,Topic_Num,Tweet_Num,Topic_Perc_Contrib,Keywords,Representative Text
0,0.0,571,0.9117,"health, fight, make, pandem, mask, time, say, help, face, think","[keep, think, master, public, health, write, doctor, dissert, global, effort, tackl, aid, pandem]"
1,1.0,145,0.9376,"peopl, test, coronavirus, like, help, state, keep, everyon, home, virus","[coronavirus, affect, everyon, discrimin, basi, religion, cast, rich, poor, either]"
2,2.0,861,0.9332,"case, work, coronavirus, speak, natur, caus, father, murder, poverti, racism","[natur, caus, father, murder, work, racism, poverti, speak, trut]"
